## Docker como maquina virtual ligera para ejecutar contenedor MySql

Docker actúa como una máquina virtual ligera que ejecuta el contenedor de MySQL de forma aislada. El contenedor tiene todo lo necesario para ejecutar MySQL, incluyendo el servidor de bases de datos, y no depende de que tengas MySQL instalado en tu sistema operativo local.  

¿Qué hace Docker en este caso?  
Aísla el contenedor: Cuando ejecutas el contenedor mysql-employees, Docker ejecuta una versión de MySQL dentro del contenedor sin necesidad de que tú tengas MySQL instalado en tu equipo.  

Usa el volumen montado: Al especificar un volumen con -v C:/Users/User/OneDrive/Escritorio/data:/var/lib/mysql, estás diciéndole a Docker que almacene los datos de la base de datos en tu máquina local, pero el servidor MySQL sigue funcionando dentro del contenedor.  

Acceso al contenedor: Puedes acceder a la base de datos MySQL mediante comandos de Docker, como el comando docker exec para interactuar con MySQL, sin tener que tener MySQL en tu máquina.

Voy a probar con una base de datos de ejemplo en docker.    
1. Entro a  DockerHub  
https://hub.docker.com/r/genschsa/mysql-employees  
2. (Windows) Ejecuto:  

In [ ]:
docker pull genschsa/mysql-employees

3. Después:

In [ ]:
docker run -d --name mysql-employees -p 3306:3306 -e MYSQL_ROOT_PASSWORD=college -v $(pwd)/data:/var/lib/mysql genschsa/mysql-employees

4. Para verificar que el contenedor está corriendo:

In [ ]:
docker ps

5. Conectar a MySQL: (te pide contraseña, la que se puso en el comando anterior)

In [ ]:
docker exec -it mysql-employees mysql -u root -p

Ya está, todos los datos de MySQL estarán guardados en esa carpeta en tu máquina local. Puedes acceder a ellos directamente desde tu sistema de archivos para ver y modificar la base de datos si es necesario.

Para ver logs del contenedor:

In [ ]:
docker logs mysql-employees

Detener el contenedor:

In [ ]:
docker stop mysql-employees

Eliminar el contenedor:

In [ ]:
docker rm mysql-employees


## Raw connection to Database 

In [35]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/16.1 MB 11.8 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/16.1 MB 11.2 MB/s eta 0:00:02
   ---------------- ----------------------- 6.6/16.1 MB 10.9 MB/s eta 0:00:01
   ---------------------- ----------------- 8.9/16.1 MB 11.1 MB/s eta 0:00:01
   ---------------------------- ----------- 11.3/16.1 MB 11.2 MB/s eta 0:00:01
   ----------------------------------- ---- 14.2/16.1 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 11.1 MB/s eta 0:00:00


Creamos un conector y un cursor para ejecutar las consultas:  
Un cursor es un objeto que permite interactuar con los datos de una base de datos. Se usa para ejecutar consultas SQL y recorrer los resultados de esas consultas. Con el cursor, puedes obtener, modificar o manipular datos de manera controlada.

In [39]:
import mysql.connector

connection = mysql.connector.connect(
    user='root',
    password='college',
    host='localhost',
    database='employees',
    ssl_disabled=True
)
cursor = connection.cursor()

Creamos un objeto query que es un String que contendrá el query SQL:

In [41]:
query = """
    SELECT *
    FROM employees e
    JOIN salaries s
    ON e.emp_no = s.emp_no
    WHERE e.hire_date > '1999-01-01';
"""

Ejecutamos el query con el cursor:

In [43]:
cursor.execute(query)

Este cursor nos dará linea por linea, entonces para extraer los datos hay que iterar sobre el query.  
Enumeramos sobre el cursor y rompemos en la primera vuelta para no mostrar todo.

In [49]:
for i, data in enumerate(cursor):
    break

In [ ]:
data

Todo junto, esta vez almacenando todos los resultados en un array de objetos y cerramos el cursor y la conexión.

In [61]:
import mysql.connector

connection = mysql.connector.connect(
    user='root',
    password='college',
    host='localhost',
    database='employees',
    ssl_disabled=True
)
cursor = connection.cursor()
query = """
    SELECT *
    FROM employees e
    JOIN salaries s
    ON e.emp_no = s.emp_no
    WHERE e.hire_date > '1999-01-01';
"""
cursor.execute(query)
results = []
for i, data in enumerate(cursor):
    results.append(data)
cursor.close()
connection.close()

In [65]:
len(results)

4853

Estos 4 mil resultados los podemos guardar en un Dataframe de Pandas:  
Hay que darles nombre a las columnas.

In [68]:
import pandas as pd

df_pulled = pd.DataFrame(results)
df_pulled.columns = ['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date', 'empl_no.1', 'salary', 'from_date', 'to_date']
df_pulled.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,empl_no.1,salary,from_date,to_date
0,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,44276,1999-04-30,2000-04-29
1,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46946,2000-04-29,2001-04-29
2,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46775,2001-04-29,2002-04-29
3,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,50032,2002-04-29,9999-01-01
4,10105,1962-02-05,Hironoby,Piveteau,M,1999-03-23,10105,59258,1999-05-17,2000-05-16
